In [1]:
# 加载数据函数
# 数据标准化
# 计算协方差矩阵的特征值和单位特征向量
# PCA处理主函数
import numpy as np
import pandas as pd
import time

In [1]:
def load_data(filepath):
    '''
    输入：文件路径, csv格式，输入数据包含特征标签
    输出：data（array）
    '''
    with open(filepath) as f:
        next(f)
        data = np.array([line.strip('\n').split(',') for line in f], dtype=float)
    return data

def normalize(X):
    X_mean = np.mean(X, axis=0)
    X_std = np.std(X, axis=0)
    for i in range(len(X)):
        X[i] = (X[i] - X_mean) / (X_std + 3.14e+12)
    
    return X
    
    
def cal_SVD(X):
    '''
    输入：样本数据array
    输出：协方差矩阵的特征值，特征向量
    '''
    new_X = X.T / np.sqrt(len(X) - 1)
    eigenvalue, v = np.linalg.eig(np.matmul(new_X.T, new_X)) # 计算协方差矩阵的特征值和特征向量，注意这里的特征向量并不一定是单位的，所以还需要单位化
    temp =  {} # 临时的字典用来保存特征值和特征向量对，其中key为特征值，value为特征向量
    for i in range(len(eigenvalue)):
        temp[eigenvalue[i]] = v[i]
    # 对特征向量进行从大到小排序
    eigenvalue = sorted(temp, reverse=True) # 将字典传入sorted函数，输出是关于字典key的列表，reverse=True表示降序排列
    V = np.empty(len(eigenvalue))
    # 特征向量单位化
    for i in eigenvalue:
        d = 0
        for j in range(len(temp[i])):
            d += temp[i][j] ** 2
        V.append(temp[i] / np.sqrt(d))
    return eigenvalue, V

def PAC(X, k):
    '''
    输入：X，k降维后的维度
    输出：降维后的数据
    '''
    X = normalize(X)
    eigenvalue, V = cal_SVD(X)
    Y = np.matmul(V[:, 0:k].T, X)
    
    return Y